In [114]:
#h is height of the tree, assume log(n), where n is the number of nodes
class Binary_Node:
    # node 
    def __init__(A,x):
        A.item = x
        A.left = None
        A.right = None
        A.parent = None

    # O(n): inorder traversal
    def subtree_iter(A):
        if A.left: yield from A.left.subtree_iter()
        yield A
        if A.right: yield from A.right.subtree_iter()
    
    # O(h): return the left most node in the subtree, assume the largest
    def subtree_first(A):
        if A.left: return A.left.subtree_first()
        else: return A
    
    # O(h): reuturn the right most node in the subtree, assume the smallest
    def subtree_last(A):
        if A.right: return A.right.subtree_last()
        else: return A

    # O(h) return the successor
    def successor(A):
        if A.right: return A.right.subtree_first()
        # find the lowest ancestor of <A> such that <A> is in the ancestor left tree (while loop stop when A is not A.parent.right)
        while A.parent and (A is A.parent.right):
            A = A.parent
        # we need to return A.parent since <A> is currently a root of subtree
        return A.parent

    # O(h) return the predecesor
    def predecessor(A):
        if A.left: return A.left.subtree_last()
        # find the the lowest ancestor of <A> such that <A> is in the ancestor right tree (while loop stop when A is not A.parent.left)
        while A.parent and (A is A.parent.left):
            A = A.parent
        # we need to return A.parent since <A> is currently a root of subtree
        return A.parent
    
    # O(h) insert node B before a given node A
    def subtree_insert_before(A, B):
        # insert B to the right of the right most node in A.left subtree
        if A.left:
            A = A.left.subtree_last()
            A.right, B.parent = B, A
        else:
            A.left, B.parent = B, A
    
    # O(h) insert node B after a given node A
    def subtree_insert_after(A, B):
        # insert B to the left of the left most node in A.right subtree
        if A.right:
            A = A.right.subtree_first()
            A.left, B.parent = B, A
        else:
            A.right, B.parent = B, A
    
    # O(h) delete node while maintain the order
    def subtree_delete(A):
        if A.left or A.right:
            if A.left: B = A.predecessor()
            else: B = A.successor()
            A.item, B.item = B.item, A.item
            return B.subtree_delete()
        if A.parent:
            if A.parent.left is A: A.parent.left = None
            else: A.parent.right = None
        return A

In [115]:
# Top-Level Data Structure
class Binary_Tree:
    def __init__(T, Node_Type = Binary_Node):
        T.root = None
        T.size = 0
        T.Node_Type = Node_Type

    def __len__(T): return T.size()

    def __iter__(T):
        for A in T.root.subtree_iter():
            yield A.item

    # Build Binary Tree from array that preserve the extrinsic order of array      
    def build(self, X):
        # A is a coppied array of X
        A = [x for x in X]
        # build a subtree of array contains A[i] to A[j]
        def build_subtree(A, i, j):
            c = (i + j) // 2            # find the middle
            root = self.Node_Type(A[c]) # == Binary_Node(A[middle])
            if i < c:
                root.left = build_subtree(A, i, c - 1)
                root.left.parent = root
            if c < j:
                root.right = build_subtree(A, c + 1, j)
                root.right.parent = root
            # base case when i >= c and c >= j
            return root
        self.root = build_subtree(A, 0, len(A) - 1)


Binary Search Tree (Set)

In [116]:
# inheritence
class BST_Node(Binary_Node):
    
    # O(h) find base on key attribute of A.item object
    def subtree_find(A, k):
        if k < A.item.key:
            if A.left: return A.left.subtree_find(k)
        elif k > A.item.key:
            if A.right: return A.right.subtree_find(k)
        else:
            return A
        return None

    # O(h) find the nearest key after k
    def subtree_find_next(A, k):
        if A.item.key <= k:
            if A.right: return A.right.subtree_find_next(k)
            else: return None
        elif A.left:
            B = A.left.subtree_find_next(k)
            if B: return B
        return A
    
    # O(h) 
    def subtree_find_prev(A, k):
        if A.item.key >= k:
            if A.left: return A.left.subtree_find_prev(k)
            else: return None
        elif A.right:
            B = A.right.subtree_find_prev(k)
            if B: return B
        return A

    # O(h) 
    def subtree_insert(A, B):
        if B.item.key < A.item.key:
            if A.left: A.left.subtree_insert(B)
            else: A.subtree_insert_before(B)
        elif B.item.key > A.item.key:
            if A.right: A.right.subtree_insert(B)
            else: A.subtree_insert_after(B)
        else:
            A.item = B.item

class Set_Binary_Tree(Binary_Tree):
    def __init__(self): super().__init__(BST_Node)

    def iter_order(self): yield from self

    def build(self, X):
        for x in X: self.insert(x)
    
    def find_min(self):
        if self.root: return self.root.subtree_first().item

    def find_max(self):
        if self.root: return self.root.subtree_last().item

    def find(self, k):
        if self.root:
            node = self.root.subtree_find(k)
            if node: return node.item
    
    def find_next(self, k):
        if self.root:
            node = self.root.subtree_find_next(k)
            if node: return node.item
        
    def find_prev(self, k):
        if self.root:
            node = self.root.subtree_find_prev(k)
            if node: return node.item

    def insert(self, x):
        new_node = self.Node_Type(x)
        if self.root:
            self.root.subtree_insert(new_node)
            if new_node.parent is None: return False
        else:
            self.root = new_node
        self.size += 1
        return True

    def delete(self, k):
        assert self.root
        node = self.root.subtree_find(k)
        assert node
        ext = node.subtree_delete()
        if ext.parent is None: self.root = None
        self.size -= 1
        return ext.item


